In [1]:
import numpy as np
from pycococreatortools import pycococreatortools
import os
import glob
import cv2
from tqdm import tqdm
import json

In [2]:
home_path=os.path.expanduser('~')
dir0_with_images_and_ann = os.path.join(home_path, 'Pobrane/images/Weapon-images')
dir1_with_images_and_ann = os.path.join(home_path, 'Pobrane/images/Weapon-images-outdoor')

In [3]:
def get_segmentation(in_dict):
    segmentation = []
    for contours_pack in in_dict['contours']:
        for contour in contours_pack:
            segmentation.append(contour['x'])
            segmentation.append(contour['y'])
    return segmentation

def get_bbox(in_dict):
    left = in_dict['bbox']['left']
    top = in_dict['bbox']['top']
    right = in_dict['bbox']['right']
    bottom = in_dict['bbox']['bottom']
    width = right-left
    height = bottom-top
    return [left, top, width, height]

In [4]:
cats_ids = {"person":0,"weapon":1, 'People with weapons':0}
image_id = 0
coco_output = {'images': [], "annotations": [], 'type': "instances"}
coco_output['categories'] = [{"supercategory": "person","id": cats_ids['person'],"name": "person"},
    {"supercategory": "weapon","id": cats_ids['weapon'],"name": "weapon"}]
for src_dir in [dir0_with_images_and_ann, dir1_with_images_and_ann]:
    images_pathes = glob.glob(src_dir+'/*.png')
    for image_path in tqdm(images_pathes):
        image_id += 30
        if image_id !=30:
            continue
        image = cv2.imread(image_path)
        parent_dir, image_name  = image_path.split('/')[-2:]
        image_filename_with_parentdir = os.path.join(parent_dir, image_name)
        image_info = pycococreatortools.create_image_info(
        image_id, image_filename_with_parentdir, image.shape)
        coco_output["images"].append(image_info)
        cutted_image_path = '_'.join(image_path.split('_')[:-1])
        annotation_list = glob.glob(cutted_image_path+'*.json')
        
        for i, annot_path in enumerate(annotation_list):
            with open(annot_path, 'r') as f:
                desc = json.load(f)
                if desc['tagName'] not in cats_ids:
                    print(desc['tagName'])
                    continue
                annotation_info = {}
                annotation_info['segmentation'] = get_segmentation(desc)
                annotation_info['area'] = desc['area']
                annotation_info['iscrowd'] = 0
                annotation_info['image_id'] = image_id
                annotation_info['bbox'] = get_bbox(desc)
                annotation_info['category_id'] = cats_ids[desc['tagName']]
                annotation_info['id'] =  image_id + i
                coco_output['annotations'].append(annotation_info)

100%|██████████| 9998/9998 [00:00<00:00, 2827118.68it/s]

mouse
mouse
book
laptop
table
book
table
table


In [5]:
annotation_file='/home/m/Pobrane/trainval.json'
os.makedirs(os.path.split(annotation_file)[0], exist_ok=True)
with open(annotation_file, 'w') as f:
    json.dump(coco_output, f)